# PicklSeq & Haplotype Caller


## Introduction

This is a notebook is the implementation of the methods introduced in **"Machine learning for targeted denoising and haplotype phasing of mixed clone pathogens using nanopore sequence data"**. It combines the PicklSeq and Haplotype caller library. The PicklSeq tool is a high-level wrapper that combines several open source tools to extract, map, and align raw fastq files into Python-friendly .pkl format for subsequent processing.

#### PicklSeq:
https://github.com/paopaoch/PicklSeq

#### VariantCalling
https://github.com/paopaoch/VariantCalling

## Section 1: PicklSeq

In this section, we set up the environment on Colab to run PicklSeq. This includes the packages and open-source tools installation (SamTools, Minimap2, and Chopper). Lastly, we clone the PicklSeq repository and complete the envrionment setup.

### Installing packages and tools

In [1]:
######### Force Environment to use Keras < 3.0 #########
!pip install "keras<3.0.0" "tensorflow<2.16" "tf-models-official<2.16" mediapipe-model-maker

######### Packages Installation #########
!apt-get install autoheader
!apt-get install autoconf

######### SamTools #########
# Install HTSlib
!git clone https://github.com/samtools/htslib --recursive
%cd htslib/
!autoreconf -i  # Build the configure script and install files it uses
!./configure    # Optional but recommended, for choosing extra functionality
!make
!make install

%cd /content
!git clone https://github.com/samtools/samtools --recursive
%cd /content/samtools/
!echo "Running autoheader"
!pwd
!autoheader            # Build config.h.in
!autoconf -Wno-syntax  # Generate the configure script
!./configure           # Needed for choosing optional functionality
!make
!make install

######### MiniMap2 #########
%cd /content/
!curl -L https://github.com/lh3/minimap2/releases/download/v2.26/minimap2-2.26_x64-linux.tar.bz2 | tar -jxvf -
!cp /content/minimap2-2.26_x64-linux/minimap2 /usr/local/bin

######### Chopper #########
%cd /content/
!wget https://github.com/wdecoster/chopper/releases/download/v0.6.0/chopper-linux.zip
!yes|unzip chopper-linux.zip
!cp /content/chopper /usr/local/bin
!chmod +x /usr/local/bin/chopper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tensorflow-metadata to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

### Cloning the PicklSeq Repo and Downloading the Example Fastq File

In [4]:
%cd /content/
!git clone https://github.com/cchuenchoksan/PicklSeq.git

!gdown https://drive.google.com/uc?id=1scYGpLgL3Aj0d6MYoT-VaWasBx0XY0qB # mix_3clones.zip
!gdown https://drive.google.com/uc?id=1huSbAzYVNKOfQg7mgXIBnrvGeXLjdj2J # single_sample.zip
!gdown https://drive.google.com/uc?id=1kgvCYVxmIg-JJteAb0GC6Z_JconQ4Y9U # mix_2clones.zip

!yes | unzip -j mix_3clones.zip
!yes | unzip -j mix_2clones.zip
!yes | unzip -j single_samples.zip

/content
Cloning into 'PicklSeq'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 89 (delta 0), reused 1 (delta 0), pack-reused 86 (from 1)
Receiving objects: 100% (89/89), 47.85 MiB | 37.38 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Downloading...
From (original): https://drive.google.com/uc?id=1scYGpLgL3Aj0d6MYoT-VaWasBx0XY0qB
From (redirected): https://drive.google.com/uc?id=1scYGpLgL3Aj0d6MYoT-VaWasBx0XY0qB&confirm=t&uuid=aea77b08-504d-45ea-b78f-33caf8ddac5e
To: /content/mix_3clones.zip
100% 826M/826M [00:11<00:00, 72.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1huSbAzYVNKOfQg7mgXIBnrvGeXLjdj2J
From (redirected): https://drive.google.com/uc?id=1huSbAzYVNKOfQg7mgXIBnrvGeXLjdj2J&confirm=t&uuid=db67b849-61c9-4ebd-b34c-6425cc487d2e
To: /content/single_samples.zip
100% 551M/551M [00:03<00:00, 169MB/s]
Downloading...
From (original): https://drive.google.com

### Running PickleSeq

In this example we will be supplying a fastq file containing read data of *P. falciparum* DD2 clones. We will be aligning and matching the chloroquine resistance transporter (CRT) sequence. Successful execution of the program will generate an output pickle file in the /content directory

Initializing Python variables. We will need these variables for both PicklSeq and Haplotype Caller

## Here we start the benchmarking

In [5]:
import time
time_start = time.time()

### Select Sequence Here
Supported sequences: pfcrt (CRT), pfdhps (DHPS), pfdhfr (DHFR), SARS-Cov2 RBM (RBM200)

In [6]:
# User Input
sequence_selected = "CRT" # "CRT" | "DHPS" | "DHFR" | "RBM200"
barcode = "02" # 00-09

In [7]:
if sequence_selected == "CRT":
    img_row, seq_length = 101, 178
    clone_names = ["3D7","DD2","7G8"]
    truncation_range = [[0,178]]
    nb_mutations = len(clone_names)
    clone_file = "crt_clones.txt"
    model_name = "Comparator_CRT.keras"
elif sequence_selected == "DHPS":
    img_row, seq_length = 101, 642
    clone_names = ["3D7","DD2","HB3"]
    truncation_range = [[70,130],[600,642]]
    nb_mutations = len(clone_names)
    clone_file = "dhps_clones.txt"
    model_name = "Comparator_DHPS_truncate.keras"
elif sequence_selected == "DHFR":
    img_row, seq_length = 101, 491
    clone_names = ["3D7","7G8","DD2","HB3"]
    truncation_range = [[100,175],[280,350]]
    nb_mutations = len(clone_names)
    clone_file = "dhfr_clones.txt"
    model_name = "Comparator_DHFR_truncate.keras"
elif sequence_selected == "RBM200":
    img_row, seq_length = 101, 200
    clone_names = ["WUHAN","ALPHA","DELTA","OMICRON"]
    truncation_range = [[0,200]]
    nb_mutations = len(clone_names)
    clone_file = "rbm_clones_200.txt"
    model_name = "Comparator_SARS-Cov2.keras"

In [8]:
sequence_selected = "CRT"
seq_length = 178
clone_names = ["3D7","DD2","7G8"]
nb_mutations = len(clone_names)
clone_file = "crt_clones.txt"

In [9]:
import os
import subprocess

%cd /content/PicklSeq/
# Define the sequence
in_file = r"/content/FAY33695_pass_barcode" + barcode + ".fastq"

out_file = in_file.replace(".fastq",".pkl")
print("Currently processing: ", in_file, "\t expecting output: ", out_file)
subprocess.run(["python", "picklseq.py", "-f="+in_file, "-o="+out_file, "-c=20", "-M=4000", "-t="+sequence_selected])

%cd /content


/content/PicklSeq
Currently processing:  /content/FAY33695_pass_barcode02.fastq 	 expecting output:  /content/FAY33695_pass_barcode02.pkl
/content


## Section 2: Haplo-Reader

In this section, we will perform haplotype-calling on the extracted reads from fastq file from PicklSeq. We start off with the loading of the ML model, and subsequently generate the similarity score with the reference sequences (considering the 3D7,DD2, and 7G8 clones only). Each read will be matched to the reference clone with the highest similarity score with a threshold of 0.5, when no reference sequences achieve score > 0.5, the read will be matched to the unknown group. This is done in a read-by-read manner until all read data in the pickle file has been processed. As the last step of the haplotype calling process, the frequency / proportion of each clone is then computed.

### Cloning the Repository

In [10]:
%cd /content/
!git clone https://github.com/paopaoch/VariantCalling.git


/content
Cloning into 'VariantCalling'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 614 (delta 115), reused 161 (delta 94), pack-reused 419 (from 1)
Receiving objects: 100% (614/614), 131.25 MiB | 32.26 MiB/s, done.
Resolving deltas: 100% (308/308), done.
Updating files: 100% (149/149), done.


#### Running Inference on the Preprocessed Pickle File

In [11]:
%cd /content/VariantCalling
import tensorflow as tf
import VariantCalling as vc
import Comparator
import pickle
import numpy as np
import pandas as pd
import importlib
importlib.reload(Comparator)

model_path = r"/content/VariantCalling/comparator_models/" + model_name
model = Comparator.load_comparator(model_path)
tracker, pred_list = Comparator.run_comparator_inference(model,out_file,clone_file)
print("\nClone\t\tProportion")
for i in range(len(tracker)-1):
    print(clone_names[i] + ":\t\t" + str(round(tracker[i]*100/sum(tracker),2)) + "%")
print("Unknown:\t" + str(round(tracker[len(tracker)-1]*100/sum(tracker),2)) + "%")
print(in_file)
time_end = time.time()
print("Time elapsed (s):\t",time_end - time_start)

/content/VariantCalling
2400/2400 [==============================] - 65s 27ms/step

Clone		Proportion
3D7:		97.62%
DD2:		0.97%
7G8:		0.69%
Unknown:	0.72%
/content/FAY33695_pass_barcode02.fastq
Time elapsed (s):	 582.7032446861267


# Expected Results
## CRT

| ont_barcode| 3D7 + HB3 |	DD2 |
| -------- | ------- | -------- |
| barcode01| 0.99 | 0.01 |
| barcode02| 1.00 | 0.00 |
| barcode03| 0.05 | 0.95 |
| barcode04| 0.11 | 0.89 |
| barcode05| 0.10 | 0.90 |
| barcode06| 0.39 | 0.61 |
| barcode07| 0.46 | 0.54 |
| barcode08| 0.40 | 0.60 |
| barcode09| 0.45 | 0.55 |

## DHPS
| ont_barcode | 3D7 + HB3 | DD2 |
| -------- | ------- | -------- |
| barcode01 | 0.988 |	0.012 |
| barcode02 | 0.988 |	0.012 |
| barcode03 | 0.012 |	0.988 |
| barcode04 | 0.255 |	0.745 |
| barcode05 | 0.283 |	0.717 |
| barcode06 | 0.842 |	0.158 |
| barcode07 | 0.842 |	0.158 |
| barcode08 | 0.446 |	0.554 |
| barcode09 | 0.503 |	0.497 |

## DHFR
| ont_barcode | 3D7 | DD2 | HB3 |
| -------- | ------- | -------- | -------- |
| barcode01 | 0.985 | 0.003 | 0.012 |
| barcode02 | 0.011 | 0.004 | 0.986 |
| barcode03 | 0.015 | 0.923 | 0.062 |
| barcode04 | 0.251 | 0.704 | 0.045 |
| barcode05 | 0.248 | 0.698 | 0.054 |
| barcode06 | 0.81 | 0.168 | 0.022 |
| barcode07 | 0.819 | 0.16 | 0.021 |
| barcode08 | 0.337 | 0.526 | 0.137 |
| barcode09 | 0.359 | 0.508 | 0.133 |